In [29]:
import pandas as pd
import numpy as np

In [77]:
# Read dataset and add columns names
# I think in the instructions email you forgot to mention one variable named "contract"
scp1205 = pd.read_csv('Medicare_Advantage/scp-1205.csv', names=['countyname', 'state', 'contract', 'healthplanname', 'typeofplan', 'countyssa', 'eligibles', 'enrollees', 'penetration', 'ABrate'], header=None,)

In [78]:
scp1205.head()

,countyname,state,contract,healthplanname,typeofplan,countyssa,eligibles,enrollees,penetration,ABrate
0,AUTAUGA,AL,H0150,HEALTHSPRING OF ALABAMA INC.,HMO,1000,6883,313,4.55,654.22
1,AUTAUGA,AL,H0151,UNITED HEALTHCARE OF ALABAMA INC.,HMO,1000,6883,12,0.17,654.22
2,AUTAUGA,AL,H0154,VIVA HEALTH INC.,HMO,1000,6883,181,2.63,654.22
3,AUTAUGA,AL,H1804,HUMANA INSURANCE COMPANY,PFFS,1000,6883,23,0.33,654.22
4,AUTAUGA,AL,90091,UNITED MINE WORKERS OF AMERICA,HCPP,1000,6883,,,654.22


In [80]:
scp1205.describe()

,countyname,state,contract,healthplanname,typeofplan,countyssa,eligibles,enrollees,penetration,ABrate
count,38013,38013,38013,38013,38013,38013,38013,38013,38013,38013
unique,1864,57,437,330,11,3176,2821,1557,1184,1133
top,JEFFERSON,FL,H1804,HUMANA INSURANCE COMPANY,HMO,3060,436805,,,591.91
freq,381,3049,1771,1995,20602,177,177,30908,30974,10149


In [81]:
scp1205.dtypes

countyname        object
state             object
contract          object
healthplanname    object
typeofplan        object
countyssa         object
eligibles         object
enrollees         object
penetration       object
ABrate            object
dtype: object

In [82]:
# County-plan pairs that have missing values for eligibles, enrollees, and penetration should be considered as 
# having zero values for these variables.

# Convert 'eligibles', 'enrollees', and 'penetration' to numeric, forcing non-numeric values to NaN
scp1205['eligibles'] = pd.to_numeric(scp1205['eligibles'], errors='coerce')
scp1205['enrollees'] = pd.to_numeric(scp1205['enrollees'], errors='coerce')
scp1205['penetration'] = pd.to_numeric(scp1205['penetration'], errors='coerce')

# Fill missing values with 0
scp1205['eligibles'].fillna(0, inplace=True)
scp1205['enrollees'].fillna(0, inplace=True)
scp1205['penetration'].fillna(0, inplace=True)

In [83]:
scp1205.head()

,countyname,state,contract,healthplanname,typeofplan,countyssa,eligibles,enrollees,penetration,ABrate
0,AUTAUGA,AL,H0150,HEALTHSPRING OF ALABAMA INC.,HMO,1000,6883.0,313.0,4.55,654.22
1,AUTAUGA,AL,H0151,UNITED HEALTHCARE OF ALABAMA INC.,HMO,1000,6883.0,12.0,0.17,654.22
2,AUTAUGA,AL,H0154,VIVA HEALTH INC.,HMO,1000,6883.0,181.0,2.63,654.22
3,AUTAUGA,AL,H1804,HUMANA INSURANCE COMPANY,PFFS,1000,6883.0,23.0,0.33,654.22
4,AUTAUGA,AL,90091,UNITED MINE WORKERS OF AMERICA,HCPP,1000,6883.0,0.0,0.00,654.22


In [94]:
# Exclude territories such at Puerto Rico and Guam
scp1205['state'] = scp1205['state'].str.strip() # Strip any leading/trailing spaces from the 'state' column
scp1205 = scp1205.loc[~scp1205['state'].isin(['PR', 'GU'])]

In [96]:
scp1205.describe()

,eligibles,enrollees,penetration
count,3.710800e+04,37108.00000,37108.000000
mean,5.782473e+04,161.77040,0.396037
std,1.117709e+05,1822.03532,2.086715
min,0.000000e+00,0.00000,0.000000
25%,6.120500e+03,0.00000,0.000000
50%,1.803700e+04,0.00000,0.000000
75%,6.401600e+04,0.00000,0.000000
max,1.099158e+06,149101.00000,54.380000


In [99]:
# Create conditions for the two plan counts
scp1205['qualified_enrollees'] = scp1205['enrollees'] > 10
scp1205['qualified_penetration'] = scp1205['penetration'] > 0.5

/var/folders/6_/w9g0lyhn5632jr666js3y_lw0000gn/T/ipykernel_22996/3892209551.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scp1205['qualified_enrollees'] = scp1205['enrollees'] > 10
/var/folders/6_/w9g0lyhn5632jr666js3y_lw0000gn/T/ipykernel_22996/3892209551.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scp1205['qualified_penetration'] = scp1205['penetration'] > 0.5


In [101]:
# Group by the relevant columns and calculate the aggregates
county_data = scp1205.groupby(['countyname', 'state', 'countyssa', 'eligibles'], as_index=False).agg(
    numberofplans1=('qualified_enrollees', 'sum'),  # Count plans with > 10 enrollees
    numberofplans2=('qualified_penetration', 'sum'),  # Count plans with penetration > 0.5%
    totalenrollees=('enrollees', 'sum')  # Total enrollees per county
)

In [102]:
county_data

,countyname,state,countyssa,eligibles,numberofplans1,numberofplans2,totalenrollees
0,ABBEVILLE,SC,42000,4062.0,1,1,291.0
1,ACADIA,LA,19000,9537.0,2,1,94.0
2,ACCOMACK,VA,49000,7169.0,0,0,0.0
3,ADA,ID,13000,39898.0,10,3,8203.0
4,ADAIR,IA,16000,1589.0,1,1,29.0
...,...,...,...,...,...,...,...
3092,YUKON KOYUKUK,AK,2290,486.0,0,0,0.0
3093,YUMA,AZ,3130,23200.0,12,1,1473.0
3094,YUMA,CO,6620,1667.0,1,1,28.0
3095,ZAPATA,TX,45982,1451.0,0,0,0.0


In [103]:
# Calculate total penetration for each county
county_data['totalpenetration'] = 100 * county_data['totalenrollees'] / county_data['eligibles']

# Sort the data by state and county
county_data.sort_values(by=['state', 'countyname'], inplace=True)

In [104]:
county_data

,countyname,state,countyssa,eligibles,numberofplans1,numberofplans2,totalenrollees,totalpenetration
2754,UNDER-11,,01xxx,0.0,1,0,979.0,inf
2755,UNDER-11,,02xxx,0.0,1,0,150.0,inf
2756,UNDER-11,,03xxx,0.0,1,0,1500.0,inf
2757,UNDER-11,,04xxx,0.0,1,0,904.0,inf
2758,UNDER-11,,05xxx,0.0,1,0,4155.0,inf
...,...,...,...,...,...,...,...,...
2653,SWEETWATER,WY,53180,4026.0,2,2,184.0,4.570293
2688,TETON,WY,53190,1698.0,0,0,0.0,0.000000
2750,UINTA,WY,53200,2006.0,1,1,62.0,3.090728
2908,WASHAKIE,WY,53210,1554.0,0,0,0.0,0.000000
